# My Test2
## test 1
* <code>optimizer.step()</code>에 따라 모델의 가중치를 실제로 뽑아서 변하는지 확인하기
    * optimizer에 담긴 parameter가 변하는 것인지 or 실제 모델의 가중치가 변하는 것인지 확인하기 위해서
## test 2
* 또한 <code>loss.backward()</code>를 할 때 원리가 무엇인지 <code>grad_fn</code>과 관련된 것인지 분석

In [68]:
import torch
from torch.utils.data import DataLoader
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor

In [69]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

train_loader = DataLoader(training_data, batch_size=16)
test_loader = DataLoader(test_data, batch_size=16)

In [80]:
class MyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(28*28, 512)
        self.linear2 = nn.Linear(512, 10)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        
        return x

In [81]:
from torchsummary import summary

model = MyNet()

summary(model, (28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 512]         401,920
              ReLU-3                  [-1, 512]               0
            Linear-4                   [-1, 10]           5,130
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 1.55
Estimated Total Size (MB): 1.57
----------------------------------------------------------------


# Test 1 Conclusion
* <code>optimizer.step()</code>을 통해 실제 모델에 갱신되는 것을 확인
* 실험이 끝나고서 생각해보면 당연하다. 왜냐하면, 모델을 저장하거나 불러올 때 <b>Model의 오브젝트를 저장</b>하기 때문이다.
* 모델의 Parameter를 인자로 넘기기 때문에 오해했다. <b>Call-By-Reference</b>의 형식으로 돌아가는 것으로 유추할 수 있다.

In [82]:
def train_loop(model, dataloader, loss_fn, optim):
    size = len(dataloader.dataset)
    print('[Weight Change Observation]')
    for batch, (X, y) in enumerate(dataloader):
        # Forward Propagation
        pred = model(X)
        loss = loss_fn(pred, y)
        
        print('=========================================')
        print(f'ORDER: {batch+1}')
        print(f'[Before Backward]\n{model.linear2.weight[0,0]}')
        
        # Backward Propagation
        optim.zero_grad() # init grad
        loss.backward()
        optim.step()
        
        print(f'[After Backward]\n{model.linear2.weight[0,0]}')
        
        if batch == (10 - 1):
            print('=========================================')
            print('print 10 times')
            break

In [83]:
learning_rate = 1e-1
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=learning_rate)

train_loop(model, train_loader, loss_fn, optim)

[Weight Change Observation]
ORDER: 1
[Before Backward]
0.005343694239854813
[After Backward]
0.005184032954275608
ORDER: 2
[Before Backward]
0.005184032954275608
[After Backward]
0.00509195402264595
ORDER: 3
[Before Backward]
0.00509195402264595
[After Backward]
0.004864421673119068
ORDER: 4
[Before Backward]
0.004864421673119068
[After Backward]
0.00480083841830492
ORDER: 5
[Before Backward]
0.00480083841830492
[After Backward]
0.00480083841830492
ORDER: 6
[Before Backward]
0.00480083841830492
[After Backward]
0.004487521015107632
ORDER: 7
[Before Backward]
0.004487521015107632
[After Backward]
0.004345327150076628
ORDER: 8
[Before Backward]
0.004345327150076628
[After Backward]
0.004016462713479996
ORDER: 9
[Before Backward]
0.004016462713479996
[After Backward]
0.0035967917647212744
ORDER: 10
[Before Backward]
0.0035967917647212744
[After Backward]
0.0033088999334722757
print 10 times


# Test 2
* 각 변수에 대해 <code>.grad_fn</code>를 호출하여 미분을 계산
* e.g. $\frac{\delta L}{\delta w}$
* 여기서 PyTorch는 w와 L에 관계에 대해 어떻게 알았을까.
* <b>연산 그래프</b>를 통해서 알고 있다.
* 이 연산 그래프 또한 <code>.grad_fn</code>에서 관리되고 있으며, 동적으로 생성되고, 더 이상 필요하지 않을 때마다 메모리에서 자동으로 삭제된다.

In [86]:
# Test 1과 같은 모델이지만 복잡함을 덜어내기 위해 같은 모델 구현
# 단, forward에서 grad_fn에 대한 출력문이 추가 되었다.

class MyNet2(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(28*28, 512)
        self.linear2 = nn.Linear(512, 10)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        print(f'input.grad_fn: {x.grad_fn}')
        x = self.flatten(x)
        print(f'flatten.grad_fn: {x.grad_fn}')
        x = self.linear1(x)
        print(f'linear1.grad_fn: {x.grad_fn}')
        x = self.relu(x)
        print(f'relu.grad_fn: {x.grad_fn}')
        x = self.linear2(x)
        print(f'linear2.grad_fn: {x.grad_fn}')
        
        return x

In [92]:
model2 = MyNet2()
loss_fn = nn.CrossEntropyLoss()
train_loader = DataLoader(training_data, batch_size=16)

X, y = next(iter(train_loader))
pred = model2(X)

input.grad_fn: None
flatten.grad_fn: None
linear1.grad_fn: <AddmmBackward0 object at 0x0000019353A94AF0>
relu.grad_fn: <ReluBackward0 object at 0x0000019353A94AF0>
linear2.grad_fn: <AddmmBackward0 object at 0x0000019353A94AF0>
